# Alternative ADCIRC momentum formulation 

This is an attempt to reformulate the conservative formulation of the momentum equations to avoid oscillations when coupled with DG elevation solver. The idea is to follow the DG formulation and integrate by parts, avoiding the need to compute the gradient of the surface elevation which was shown to be the cause of the oscillations.

## 1. Standard ADCIRC formulation
$ \newcommand{\area}[2]{\left \langle #1, #2 \right \rangle_{\Omega}} $
$ \newcommand{dx}[1]{\frac{\partial #1}{\partial x}} $
$ \newcommand{dy}[1]{\frac{\partial #1}{\partial y}} $ 
$ \newcommand{dt}[1]{\frac{\partial #1}{\partial t}} $

For each node $j$, the weak formulation for x and y momentum is

$$\area{\frac{\partial Q_x}{\partial t}}{\phi_j} = -\area{\dx{UQ_x}}{\phi_j} - \area{\dy{VQ_x}}{\phi_j} -
\area{F_{\tau}}{\phi_j} - \textcolor{red}{\area{gH \dx{\zeta}}{\phi_j}}.$$

$$\area{\frac{\partial Q_y}{\partial t}}{\phi_j} = -\area{\dx{UQ_y}}{\phi_j} - \area{\dy{VQ_y}}{\phi_j} -
\area{F_{\tau}}{\phi_j} - \textcolor{red}{\area{gH \dy{\zeta}}{\phi_j}}.$$

We see that there is a gradient term (barotropic pressure) $\partial \zeta / \partial x$ on the RHS.

## 2. New conservative formulation

Starting from the conservative form of the momentum conservation equations, DG uses the following formulation (but with each inner product on a single element). This avoids the elevation gradient term.

### Weak form
$$\area{\dt{w}_i}{\phi} = \area{\mathbf{F}_i}{\nabla \phi} -
\int_{\partial \Omega} \mathbf{F}_i \cdot n \phi_j \; d\Gamma + \area{s_i}{\phi}$$

where

$$
\begin{align}
w &= [\zeta, UH, VH] \equiv [\zeta, Q_x, Q_y]\\
\mathbf{F}_1 &= [Q_x, Q_y] \\
\mathbf{F}_2 &= [UQ_x + \frac{1}{2} g (H^2-h^2), VQ_x] \\
\mathbf{F}_3 &= [UQ_y, VQ_y + \frac{1}{2} g (H^2-h^2)] \\
s &= [ 0, g \zeta \dx{h} - F_{\tau}, g \zeta \dy{h} - F_{\tau} ]
\end{align}
$$


Expanding for $i = 2,3$ gives

$$
\begin{align}
\area{\dt{Q_x}}{\phi_j} &= \area{Q_x U}{\dx{\phi_j}} + \area{VQ_x}{\dy{\phi_j}}  + \area{\frac{1}{2} g(H^2-h^2)}{\dx{\phi_j}} \\
&- \int_{\partial \Omega} Q_x (\mathbf{U} \cdot \mathbf{n}) \phi_j \; ds -
\int_{\partial \Omega}  \frac{1}{2} g(H^2-h^2) n_x \phi_j \; ds \\
&- \area{F_{\tau}}{\phi_j} + \area{g\zeta\dx{h}}{\phi_j}\\
\area{\dt{Q_y}}{\phi_j} &= \area{Q_y U}{\dx{\phi_j}} + \area{VQ_y}{\dy{\phi_j}} + \area{\frac{1}{2} g(H^2-h^2)}{\dy{\phi_j}} \\
&- \int_{\partial \Omega} Q_y (\mathbf{U} \cdot \mathbf{n}) \phi_j \; ds -
\int_{\partial \Omega}  \frac{1}{2} g(H^2-h^2) n_y \phi_j \; ds \\
&- \area{F_{\tau}}{\phi_j} + \area{g\zeta\dy{h}}{\phi_j}
\end{align}
$$

### Quadrature rules

Currently, ADCIRC has no rule for edge integration, which is needed in our new formulation. The area integration rules are

#### Rule 1: nodal lumping

$$\area{\gamma}{\phi_j} \approx \frac{A_{NE_j}}{3} \gamma_j$$

where $\gamma_j$ is the nodal value.

#### Rule 2: fully consistent

$$\area{\dx{\gamma}}{\phi_j} = \sum_{n=1}^{NE_j} \frac{A_n}{3} \left (\dx{\gamma} \right )_n
= \sum_{n=1}^{NE_j} \frac{A_n}{3} \sum_{i=1}^3 \gamma_i \dx{\psi_i}$$

where $(x)_n$ is the (constant) value in element $n$, and $\psi_i$ is the *local* basis function on element $n$ with has value 1 at the $i^{th}$ local vertex.

#### Rule 3: derivative on test functions

$$\area{\gamma}{\dx{\phi_j}} = \sum_{n=1}^{NE_j} \left( \dx{\phi_j} \right)_n \int_{\Omega_n} \gamma \; dA
\approx \sum_{n=1}^{NE_j} A_n \bar{\gamma}_n \left( \dx{\phi_j} \right)_n$$



#### Rule 4: edge integration

Each global basis function spans two edges. We approximate $\gamma$ by its average across the two nodes of each edge. Since $\phi_j$ is goes from 0 to 1 from one node to another, its area under graph is simply half of the length of that edge.

$$\begin{align}
\int_{\partial\Omega} \gamma \phi_j \; ds &= \sum_{n=1}^2 \int_{E_n} \gamma \phi_j \; ds \\
                                        & \approx \sum_{n=1}^2 \bar{\gamma}_{n} \int_{E_n} \phi_j \; ds \\
&= \sum_{n=1}^2 \bar{\gamma}_{n} \cdot \frac{1}{2} L_n
\end{align}
$$


## 3. Implementation details

Using these rules, the computations of our terms will be as follow. In practice, we loop over each element, compute the integrals, and then add them to the corresponding 3 nodes of that element. In the code sections, it is assumed that we are already working on a specific element with nodes `NM1`, `NM2`, `NM3`.

In the implementation, as we work on each element $ n $, the test function term $ (\dx{\phi})_n $ will be comprised of 3 terms: `FDX1`, `FDX2`, `FDX3`, corresponding to 

$$\left(\dx{\phi_{NM1}} \right)_n, \left(\dx{\phi_{NM2}}\right)_n, \left(\dx{\phi_{NM3}}\right)_n,$$
respectively.
 
```{note}
Note that some terms require a factor of 3 in front since everything will be multipled by $3/A_n$ later in ADCIRC
```


### 3a. Advection terms

#### x-momentum
$$3\area{UQ_x}{\dx{\phi_j}} = 3\sum_n^{NE_j} \int_{\Omega_n} U Q_x \dx{\phi_j} \; dA \approx 3\sum_n A_n \bar{U}_n \bar{(Q_x)}_n \left(\dx{\phi_j} \right)_n$$

$$3\area{VQ_x}{\dy{\phi_j}} = 3\sum_n^{NE_j} \int_{\Omega_n} V Q_x \dy{\phi_j} \; dA \approx 3\sum_n A_n \bar{V}_n \bar{(Q_x)}_n \left(\dy{\phi_j} \right)_n$$

#### y-momentum
$$3\area{UQ_y}{\dx{\phi_j}} = 3\sum_n^{NE_j} \int_{\Omega_n} U Q_y \dx{\phi_j} \; dA \approx 3\sum_n A_n \bar{U}_n \bar{(Q_y)}_n \left(\dx{\phi_j} \right)_n$$

$$3\area{VQ_y}{\dy{\phi_j}} = 3\sum_n^{NE_j} \int_{\Omega_n} V Q_y \dy{\phi_j} \; dA \approx 3\sum_n A_n \bar{V}_n \bar{(Q_y)}_n \left(\dy{\phi_j} \right)_n$$

#### Code
```fortran
QX1Avg = (QX1N1 + QX1N2 + QX1N3) / 3.
QY1Avg = (QY1N1 + QY1N2 + QY1N3) / 3.

U1Avg = (U1N1 + U1N2 + U1N3) / 3.
V1Avg = (V1N1 + V1N2 + V1N3) / 3.

!! X-momentum
! Node NM1 contribution
TEMP_LV_A1 += 3 * (QX1Avg*U1Avg*FDX1 + QX1Avg*V1Avg*FDY1)

! Node NM2 contribution
TEMP_LV_A2 += 3 * (QX1Avg*U1Avg*FDX2 + QX1Avg*V1Avg*FDY2)

! Node NM3 contribution
TEMP_LV_A3 += 3 * (QX1Avg*U1Avg*FDX3 + QX1Avg*V1Avg*FDY3)

!! Y-momentum
! Node NM1 contribution
TEMP_LV_B1 += 3 * (QY1Avg*U1Avg*FDX1 + QY1Avg*V1Avg*FDY1)

! Node NM2 contribution
TEMP_LV_B2 += 3 * (QY1Avg*U1Avg*FDX2 + QY1Avg*V1Avg*FDY2)

! Node NM3 contribution
TEMP_LV_B3 += 3 * (QY1Avg*U1Avg*FDX3 + QY1Avg*V1Avg*FDY3)




```

### 3b. Barotropic pressure gradient terms

#### x-momentum
$$3\area{\frac{1}{2} g(H^2-h^2)}{\dx{\phi_j}} \approx 3\sum_{n=1}^{NE_j} A_n \frac{1}{2} g (\overline{H^2-h^2})_n \left(\dx{\phi_j} \right)_n$$

#### y-momentum
$$3\area{\frac{1}{2} g(H^2-h^2)}{\dy{\phi_j}} \approx 3\sum_{n=1}^{NE_j} A_n \frac{1}{2} g (\overline{H^2-h^2})_n \left(\dy{\phi_j} \right)_n$$

#### Code
```fortran
Hdiff1 = H1N1^2 - DPN1^2
Hdiff2 = H1N2^2 - DPN2^2
Hdiff3 = H1N2^2 - DPN2^2
HdiffAvg =( Hdiff1 + Hdiff2 + Hdiff3) / 3.

!! X-momentum
! Node NM1 contribution
TEMP_LV_A1 += 3 * 0.5*g*HdiffAvg*FDX1 

! Node NM2 contribution
TEMP_LV_A2 += 3 * 0.5*g*HdiffAvg*FDX2 

! Node NM3 contribution
TEMP_LV_A3 += 3 * 0.5*g*HdiffAvg*FDX3 

!! Y-momentum
! Node NM1 contribution
TEMP_LV_B1 += 3 * 0.5*g*HdiffAvg*FDY1 

! Node NM2 contribution
TEMP_LV_B2 += 3 * 0.5*g*HdiffAvg*FDY2 

! Node NM3 contribution
TEMP_LV_B3 += 3 * 0.5*g*HdiffAvg*FDY3 

```

### 3c. Lateral stress gradient terms

Currently we set `ESLM = 0` so these terms do not appear.

### 3d. Extra boundary terms
Here we will have to create a new subroutine to loop through the edges, compute the edge integral, and add it to the load vectors `MOM_LV_X` and `MOM_LV_Y` for each of the two nodes. We can simplify calculations by splitting it into elevation-specified and wall boundaries. The first term on the RHS vanishes for wall boundaries since the normal velocity is zero.

#### x-momentum

$$\begin{align}
\int_{\partial \Omega} \{[UQ_x + \frac{1}{2} g(H^2-h^2)] n_x + VQ_x n_y\} \phi_j \; d\Gamma &= 
\int_{\partial \Omega} Q_x (\mathbf{U} \cdot \mathbf{n}) \phi_j \; ds \\
&+ \int_{\partial \Omega}  \frac{1}{2} g(H^2-h^2) n_x \phi_j \; ds
\end{align}$$

#### y-momentum

$$
\begin{align}
\int_{\partial \Omega} \{UQ_y n_x + [\frac{1}{2} g(H^2-h^2) + VQ_y] n_y\} \phi_j \; d\Gamma &=
\int_{\partial \Omega} Q_y (\mathbf{U} \cdot \mathbf{n}) \phi_j \; ds \\
&+ \int_{\partial \Omega}  \frac{1}{2} g(H^2-h^2) n_y \phi_j \; ds
\end{align}
$$

#### Code
In the implementation, we will have to loop through the outer edges (like in DG).

```fortran
DO I = 1,Nedges
    ! Get the global edge number
    GED = ...

    ! Get the 2 nodes connected to this edge
    N1 = NEDNO(1, GED)
    N2 = NEDNO(2, GED)

    ! Get the normal vector
    NX = COSNX(GED)
    NY = SINNX(GED)

    QX1Avg = (QX1(N1) + QX1(N2)) / 2
    U1Avg = (UU1(N1) + UU1(N2)) / 2
    V1Avg = (VV1(N1) + VV1(N2)) / 2.

    Hdiff1 = H1N1^2 - DPN1^2
    Hdiff2 = H1N2^2 - DPN2^2

    integral = 0.5*XLEN(GED)*( (U1Avg*QX1Avg + 0.5*G*HdiffAvg) * NX + V1Avg*QX1Avg * NY )

    ! Node NM1 contribution
    TEMP_LV_A1 += integral

    ! Node NM2 contribution
    TEMP_LV_A2 += integral
    
END DO
```

### 3e. Bathymetric gradient terms

Let

$$ (\zeta)_n = \frac{1}{2} (\zeta_n^s + \zeta_n^{s+1})$$

#### x-momentum
$$3\area{g\zeta\dx{h}}{\phi_j} = 3\sum_{n=1}^{NE_j} \frac{A_n}{3} g (\zeta)_n \left(\dx{h} \right)_n$$

#### y-momentum
$$3\area{g\zeta\dy{h}}{\phi_j} = 3\sum_{n=1}^{NE_j} \frac{A_n}{3} g (\zeta)_n \left(\dy{h} \right)_n$$

#### Code
```fortran
EtaAvg1 = (ETA1(NM1) + ETA1(NM2) + ETA1(NM3)) / 3.
EtaAvg2 = (ETA2(NM1) + ETA2(NM2) + ETA2(NM3)) / 3.
EtaAvg = 0.5*(EtaAvg1 + EtaAvg2)

!! Bathymetric gradient
DBDX = DP(NM1)*FDX1+DP(NM2)*FDX2+DP(N3)*FDX3
DBDY = DP(NM1)*FDY1+DP(NM2)*FDY2+DP(N3)*FDY3

BathGradTermX = G*DBDX*EtaAvg
BathGradTermY = G*DBDY*EtaAvg


!! X-momentum
! Node NM1 contribution
TEMP_LV_A1 += BathGradTermX

! Node NM2 contribution
TEMP_LV_A2 += BathGradTermX

! Node NM3 contribution
TEMP_LV_A3 += BathGradTermX

!! Y-momentum
! Node NM1 contribution
TEMP_LV_B1 += BathGradTermY

! Node NM2 contribution
TEMP_LV_B2 += BathGradTermY

! Node NM3 contribution
TEMP_LV_B3 += BathGradTermY
```
